In [4]:
import os
os.chdir('../')

In [12]:
import json

annot_file = 'Data/slots_data/rhet_data_slots_cleaned_test.json'
with open(annot_file, 'r', encoding='utf-8') as f:
    lines = json.load(f)
    task_input, task_output = [], []
    for file in lines:
        line = lines[file]
        for rhet_tag in line:
            # Try training on only 1 task
            # if rhet_tag == 'identification':
            text = line[rhet_tag]['text']
            slots = line[rhet_tag]['slots']
            formatted_slot = ''
            for slot_key, slot_value in slots.items():
                formatted_slot+= '<'+slot_key+':'+slot_value+'>'
            task_output.append(text)
            task_input.append(formatted_slot)

In [15]:
import os
import json
# compile all json outputs 

# 'gpt_neo',

models = ['transformers_bert_base',
          't5_base',
          'gpt_neo',
          'flan_t5_base']

suffixes = [f'{i}' for i in range(1,7)]
prefix = 'results_greedy_'

root_path = 'checkpoints'

columns = ['Model','M1','M2', 'M3', 'M4', 'M5', 'M6']
data_dict = {k:[] for k in columns}
for model in models:
    data_dict['Model'].append(model)

    for suffix in suffixes:
        full_path = os.path.join(root_path, model, prefix+suffix+".json")
        with open(full_path, 'r') as f:
            results = json.load(f)
            data_dict[f'M{suffix}'].append(round(results['rougeL']*100,2))

import pandas as pd
df = pd.DataFrame(data=data_dict, columns=columns)
        

In [16]:
df

,Model,M1,M2,M3,M4,M5,M6
0,transformers_bert_base,27.72,30.66,26.64,29.38,45.94,30.21
1,t5_base,45.96,42.50,36.20,42.38,56.63,37.39
2,gpt_neo,31.74,30.72,25.55,30.54,37.84,26.51
3,flan_t5_base,42.70,40.74,34.79,40.88,54.97,33.65


In [32]:
# code for decoding results
models = ['transformers_bert_base',
          't5_base',
          'gpt_neo',
          'flan_t5_base']


decoding = 'sampling_topk_topp'
scores = ['rouge1','rouge2', 'rougeL','bleu','meteor','bert_score(p)','bert_score(r)','bert_score(f1)']


root_path = 'checkpoints'


for model in models:
    print('\n model: ', model)
    
    full_path = os.path.join(root_path, model, f'results_{decoding}.json')
    print('path: ', full_path)
    with open(full_path, 'r') as f:
        results = json.load(f)
    
    display_str = ' & '
    total = 0
    for score in scores:
        if score not in ['bert_score(p)','bert_score(r)']:
            total+=round(results[score]*100,2)

        display_str+=str(round(results[score]*100,2))+' & '
    display_str+= str(round((total/6),2))
    print(display_str)



 model:  transformers_bert_base
path:  checkpoints/transformers_bert_base/results_sampling_topk_topp.json
 & 39.88 & 14.24 & 27.8 & 13.1 & 32.95 & 88.44 & 87.96 & 88.19 & 36.03

 model:  t5_base
path:  checkpoints/t5_base/results_sampling_topk_topp.json
 & 52.16 & 27.03 & 37.33 & 22.07 & 45.05 & 91.4 & 90.96 & 91.17 & 45.8

 model:  gpt_neo
path:  checkpoints/gpt_neo/results_sampling_topk_topp.json
 & 39.36 & 18.94 & 26.6 & 10.0 & 42.0 & 87.15 & 90.83 & 88.94 & 37.64

 model:  flan_t5_base
path:  checkpoints/flan_t5_base/results_sampling_topk_topp.json
 & 53.65 & 29.35 & 39.21 & 25.19 & 47.41 & 91.48 & 91.28 & 91.37 & 47.7


In [ ]:
# error file: ../data/rhet_data\\205ChD_180926FoodEN.txt